In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from io import StringIO

### parse datetime data

In [ ]:
_ = """
time,other_col
01-01-2000 12:00:00,5
01-05-2000 1:00:00,3
01-12-2000 13:00:00,6
"""
df = pd.read_csv(StringIO(_))

df["time"] = pd.to_datetime(df["time"], format="%d-%m-%Y %H:%M:%S")
df

### truncate datetime to month end


In [ ]:
import datetime

# def month_end(dt):
#     if dt.month == 12:
#         next_mth_day_1 = datetime.datetime(dt.year + 1, 1, 1)
#     else:
#         next_mth_day_1 = datetime.datetime(dt.year, dt.month + 1, 1)
#     return next_mth_day_1 - datetime.timedelta(days=1)


def month_end(dt):
    next_mth_dt = dt + pd.DateOffset(months=1)
    next_mth_day_1 = next_mth_dt.replace(day=1)
    return next_mth_day_1 - datetime.timedelta(days=1)


df["eom"] = df["time"].apply(month_end)
df["eom"]

### working with timezones

In [17]:
# time zone localize: assuming naive datetime is in local time
df["time_sydney"] = df["time"].dt.tz_localize("Australia/Sydney")
df["time_singapore"] = df["time"].dt.tz_localize("Asia/Singapore")
df

,time,other_col,eom,time_sydney,time_singapore
0,2000-01-01 12:00:00,5,2000-01-31,2000-01-01 12:00:00+11:00,2000-01-01 12:00:00+08:00
1,2000-05-01 01:00:00,3,2000-05-31,2000-05-01 01:00:00+10:00,2000-05-01 01:00:00+08:00
2,2000-12-01 13:00:00,6,2000-12-31,2000-12-01 13:00:00+11:00,2000-12-01 13:00:00+08:00


In [18]:
# time zone convert to UTC then remove time zone info
df["time_sydney_utc_naive"] = (
    df["time_sydney"].dt.tz_convert("UTC").dt.tz_localize(None)
)
df

,time,other_col,eom,time_sydney,time_singapore,time_sydney_utc_naive
0,2000-01-01 12:00:00,5,2000-01-31,2000-01-01 12:00:00+11:00,2000-01-01 12:00:00+08:00,2000-01-01 01:00:00
1,2000-05-01 01:00:00,3,2000-05-31,2000-05-01 01:00:00+10:00,2000-05-01 01:00:00+08:00,2000-04-30 15:00:00
2,2000-12-01 13:00:00,6,2000-12-31,2000-12-01 13:00:00+11:00,2000-12-01 13:00:00+08:00,2000-12-01 02:00:00
